## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-03-10-47-00 +0000,wsj,"Copper Flat as Chinese Data, Dollar Weigh On S...",https://www.wsj.com/finance/commodities-future...
1,2025-11-03-10-46-32 +0000,bbc,Rail security to be reviewed after train stabb...,https://www.bbc.com/news/articles/cd7r4zwwq53o...
2,2025-11-03-10-44-02 +0000,nyt,At Least 15 Dead and Hundreds More Injured Aft...,https://www.nytimes.com/2025/11/02/world/asia/...
3,2025-11-03-10-33-38 +0000,bbc,Labour MPs back gambling tax to fight child po...,https://www.bbc.com/news/articles/cy7evm5vj1zo...
4,2025-11-03-10-30-00 +0000,wsj,Your Summer Weekend in Nantucket Is on This Ye...,https://www.wsj.com/lifestyle/travel/your-summ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
86,trump,28
150,new,12
92,will,11
10,train,10
141,war,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
89,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...,71
156,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...,69
31,2025-11-03-09-49-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...,69
134,2025-11-02-21-43-34 +0000,nyt,Will Trump’s Tariff Deal Tilt the Playing Fiel...,https://www.nytimes.com/2025/11/02/business/ec...,58
62,2025-11-03-07-09-13 +0000,bbc,Trump says he doubts US will go to war with Ve...,https://www.bbc.com/news/articles/c709rwxzx9yo...,57


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
89,71,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...
198,53,2025-11-02-14-11-04 +0000,nypost,UK pol tells country to ‘get on with your live...,https://nypost.com/2025/11/02/world-news/uk-po...
31,41,2025-11-03-09-49-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...
156,41,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...
39,36,2025-11-03-09-09-00 +0000,wsj,"U.S. Futures, Global Markets Start New Month M...",https://www.wsj.com/finance/u-s-futures-global...
180,36,2025-11-02-16-46-09 +0000,nypost,House Dem leader rails that Mamdani isn’t futu...,https://nypost.com/2025/11/02/us-news/house-de...
185,33,2025-11-02-16-11-31 +0000,bbc,"'No help, no food, no water': Hurricane-hit Ja...",https://www.bbc.com/news/articles/cpv11w8ez10o...
167,27,2025-11-02-18-36-44 +0000,latimes,Photos: Los Angeles celebrates Dodgers' first ...,https://www.latimes.com/california/story/2025-...
98,27,2025-11-03-01-25-26 +0000,startribune,"Hundreds honor Vietnam War, Secret War veteran...",https://www.startribune.com/minnesotans-commem...
161,26,2025-11-02-19-17-09 +0000,nypost,Gavin Newsom claims he ‘never had’ concerns ab...,https://nypost.com/2025/11/02/us-news/newsom-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
